In [2]:
# Final main code

import sys
import pygame
import json

from time import sleep
from game_stats import GameStats
from settings import Settings
from ship import Ship 
from bullet import Bullet
from alien import Alien
from button import Button
from difficulty import Difficulty
from scoreboard import Scoreboard
from life import Life
from power_shot import PowerShot
from explosion import Explosion
from sound import SoundFX


class AlienInvasion:
    """Model the overall class to manage the game assets and behaviors"""
    
    def __init__(self):
        """Initialize the game, create the game resources"""
        
        pygame.init()
        
        self.settings = Settings()
        self.screen = pygame.display.set_mode((0,0), pygame.FULLSCREEN)
        self.settings.screen_width = self.screen.get_rect().width
        self.settings.screen_height = self.screen.get_rect().height
        pygame.display.set_caption("Alien Invasion")
        
        # Create an instance to store the game statistics
        self.stats = GameStats(self)
        
        # Adding the ship attribute by calling Ship() instance
        self.ship = Ship(self)
        
        # Adding the groups by pygame.sprite.Group() function
        self.bullets = pygame.sprite.Group()
        self.aliens = pygame.sprite.Group()
        self.power_shots = pygame.sprite.Group()
        self.bullet_explosions = pygame.sprite.Group()
        self.power_shot_explosions = pygame.sprite.Group()
        self.ship_explosions = pygame.sprite.Group()
        
        # Adding the play button instance
        self.play_button = Button(self, "Play", 668, 407)
        
        # Adding the difficulty buttons instance
        self.difficulty_button = Difficulty(self, "Easy", "Medium", "Hard", "Extreme")
        
        # Adding the menu button instance
        self.resume_button = Button(self, "Resume", 668, 250)
        self.restart_button = Button(self, "Restart", 668, 400)
        self.quit_button = Button(self, "Quit", 668, 550)
        
        
        # Adding the scoreboard instance
        self.sb = Scoreboard(self)    
        
        # Create the first fleet of aliens
        self._create_fleet()
        
        # Create the sound effects instance
        self.soundfx = SoundFX(self)
    
    def run_game(self):
        """Start the main loop for the game"""
        
        while True:
            self._check_events()
            if self.stats.game_active:
                self.ship.update()
                self._update_bullets()
                self._update_aliens()
            self._update_screen()
    
    def _fire_bullet(self):
        """Fire the bullet"""
        
        # Fire a bullet if the number of existing bullets is lesser than the bullets_allowed
        if len(self.bullets) < self.settings.bullets_allowed:
            new_bullet = Bullet(self)
            self.bullets.add(new_bullet)
    
    def _fire_power_shot(self):
        """Fire the power shot"""
        
        # Fire a power shot due to the number of remaining power shots
        if self.stats.power_shots:
            new_power_shot = PowerShot(self)
            self.power_shots.add(new_power_shot)
            self.soundfx.power_shot_fx.play()
            
            self.stats.power_shots -= 1
            self.sb.prep_power_shots()
    
    def _create_fleet(self):
        """Create a fleet of aliens"""
        
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        
        # Determine the number of aliens can fit horizontally
        available_space_x = self.settings.screen_width - (2 * alien_width)
        number_aliens_x = available_space_x // (2 * alien_width) 
        
        # Determine the number of rows can fit on the screen
        ship_height = self.ship.rect.height
        available_space_y = self.settings.screen_height - (2.5 * alien_height) - ship_height
        number_rows = available_space_y // (1.25 * alien_height)
        
        number_of_rows = int(number_rows)
        
        # Create the first row of aliens.
        for row_number in range(number_of_rows):
            for alien_number in range(number_aliens_x):
                self._create_alien(alien_number, row_number)
    
    def _create_alien(self, alien_number, row_number):
        """Create an alien and place it in the row."""
        
        alien = Alien(self)
        alien_width, alien_height = alien.rect.size
        
        alien.x = alien_width + (2 * alien_width * alien_number)
        alien.rect.x = alien.x
        
        alien.y = alien_height + (1.25 * alien_height * row_number)
        alien.rect.y = alien.y
        
        self.aliens.add(alien)        
    
    def _check_fleet_edges(self):
        """Response appropriately if the alien hits the edges"""
        
        for alien in self.aliens.sprites():
            if alien.check_edges():
                self._change_fleet_direction()
                break
    
    def _change_fleet_direction(self):
        """Drop the entire fleet and change its direction"""
        
        for alien in self.aliens.sprites():
            alien.rect.y += self.settings.fleet_drop_speed
        self.settings.fleet_direction *= -1
    
    def _check_aliens_bottom(self):
        """Check if any alien reaches the bottom of the screen"""
        
        screen_rect = self.screen.get_rect()
        for alien in self.aliens.sprites():
            if alien.rect.bottom >= screen_rect.bottom:
                self._ship_hit()
                self.soundfx.ship_explosion_fx.play()
                break
    
    def _ship_hit(self):
        """Respond to the ship thats been hit by an alien"""
        
        if self.stats.ships_left > 0:
            
            # Decrement ship left
            self.stats.ships_left -= 1
            self.sb.prep_lives()

            # Delete all remaining bullets and aliens
            self.bullets.empty()
            self.aliens.empty()
            self.power_shots.empty()

            # Respawn the ship and the fleet
            self._create_fleet()
            self.ship.center_ship()
   
        else:
            # Display game over!
            self.ship_explosions.empty()
            self.stats.game_over = True
            self.stats.game_active = False
            pygame.mouse.set_visible(True)
            self.soundfx.game_over_fx.play()
            
    
    def _check_events(self):
        """Responses keyboard and mouse events"""
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.KEYDOWN:
                self._check_keydown_events(event)
            elif event.type == pygame.KEYUP:
                self._check_keyup_events(event)
            elif event.type == pygame.MOUSEBUTTONDOWN:
                mouse_pos = pygame.mouse.get_pos()
                self._check_play_button(mouse_pos)
                self._check_difficulty_button(mouse_pos)
                self._check_pause(mouse_pos)
    
    def _check_keydown_events(self, event):
        """Respond to keypresses."""
        
        # Move the ship to the right when pressing right arrow
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = True
        # Move the ship to the left when pressing left arrow
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = True
        # Exit the game with q press
        elif event.key == pygame.K_q:
            self._store_high_score()
            pygame.quit()
            sys.exit()
        # Fire bulelts with spacebar
        elif event.key == pygame.K_SPACE:
            self._fire_bullet()
        # Fire power shots with b press
        elif event.key == pygame.K_b:
            self._fire_power_shot()
        # Pause the game with p press
        elif event.key == pygame.K_p :
            # Only pause while ingame
            if self.stats.game_active == True:
                self.stats.game_pause = True
                self.stats.game_active = False
                pygame.mouse.set_visible(True)

    def _check_keyup_events(self, event):
        """Response to key releases"""
        
        # Stop the ship when releasing right arrow
        if event.key == pygame.K_RIGHT:
            self.ship.moving_right = False
        # Stop the ship when releasing left arrow
        elif event.key == pygame.K_LEFT:
            self.ship.moving_left = False
    
    def _check_pause(self, mouse_pos):
        """Check if the game is pause"""
        
        resume_button_clicked = self.resume_button.rect.collidepoint(mouse_pos)
        restart_button_clicked = self.restart_button.rect.collidepoint(mouse_pos)
        quit_button_clicked = self.quit_button.rect.collidepoint(mouse_pos)
        
        if resume_button_clicked:
            self.stats.game_active = True
            self.stats.game_pause = False
        
        elif restart_button_clicked:
            self._start_game()
            self.settings.initialize_dynamic_settings()
            self.sb.prep_images()
            self.stats.game_pause = False
            
        elif quit_button_clicked:
            pygame.quit()
            sys.exit()
    
    def _check_play_button(self, mouse_pos):
        """Start a new game when the player click play"""
        
        button_clicked = self.play_button.rect.collidepoint(mouse_pos)
        if button_clicked and self.stats.game_active == False:
            self._start_game()
            self.settings.initialize_dynamic_settings()
            self.sb.prep_images()
    
    def _check_difficulty_button(self, mouse_pos):
        """Choose the level of difficulty"""
        
        easy_button_clicked = self.difficulty_button.easy_rect.collidepoint(mouse_pos)
        medium_button_clicked = self.difficulty_button.medium_rect.collidepoint(mouse_pos)
        hard_button_clicked = self.difficulty_button.hard_rect.collidepoint(mouse_pos)
        extreme_button_clicked = self.difficulty_button.extreme_rect.collidepoint(mouse_pos)
        
        # Re-color the buttons after click another
        self.difficulty_button.re_color_button()
        
        if easy_button_clicked and self.stats.game_active == False:
            self.difficulty_button.easy_button_color = (105,105,105)
            self.settings.difficulty = 'easy'
       
        elif medium_button_clicked and self.stats.game_active == False:
            self.difficulty_button.medium_button_color = (105,105,105)
            self.settings.difficulty = 'medium'
        
        elif hard_button_clicked and self.stats.game_active == False:
            self.difficulty_button.hard_button_color = (105,105,105)
            self.settings.difficulty = 'hard'
            
        elif extreme_button_clicked and self.stats.game_active == False:
            self.difficulty_button.extreme_button_color = (105,105,105)
            self.settings.difficulty = 'extreme'
        
        # Update the button color after each click
        self.difficulty_button._prep_msg('Easy', 'Medium', 'Hard', 'Extreme')
    
    def _start_game(self):
        """Start the game"""
        
        # Reset the game stats
        self.stats.reset_stats()
        self.stats.game_active = True
        self.stats.game_over = False

        # Get rid of any remaining aliens or bullets
        self.aliens.empty()
        self.bullets.empty()
        self.power_shots.empty()

        # Respawn the fleet and center the ship
        self._create_fleet()
        self.ship.center_ship()
            
        # Hide mouse cursor ingame
        pygame.mouse.set_visible(False)
    
    def _update_aliens(self):
        """
        Check if the fleet is at an edge,
        then update the positions of all aliens in the fleet.
        """
        # Check if a alien reaches the edges
        self._check_fleet_edges()
        self.aliens.update()
        
        # Detect any ship and aliens collisions
        if pygame.sprite.spritecollideany(self.ship, self.aliens):
            explosion = Explosion(self, 'ship', self.ship.rect.centerx, self.ship.rect.centery)
            self.ship_explosions.add(explosion)
            # Play the ship explosion fx
            self.soundfx.ship_explosion_fx.play()
            self._ship_hit()
        
        # Update ship explosion effect
        self.ship_explosions.update()
        
        # Check if any alien reaches the bottom of the screen
        self._check_aliens_bottom()
        
    def _update_bullets(self):
        """Update position of the bullets and get rid of old bullets"""
        
        # Update bullet position
        self.bullets.update()
        # Update bullet explosion effect
        self.bullet_explosions.update()
        
        # Update power_shot position
        self.power_shots.update()
        # Update power shot explosion effect
        self.power_shot_explosions.update()
        
        # Get rid of old bullets and power shots
        for bullet in self.bullets.copy():
            if bullet.rect.bottom <= 0:
                self.bullets.remove(bullet)
         
        for power_shot in self.power_shots.copy():
            if power_shot.rect.bottom <= 0:
                self.power_shots.remove(power_shot)
                
        
        # Check for bullet and alien collisions
        self._check_bullet_alien_collisions()
        self._check_power_shot_alien_collisions()
        
    
    def _check_bullet_alien_collisions(self):
        """Check for bullet and alien collisions"""
        
        # Delete any bullet and alien that has collided
        collisions = pygame.sprite.groupcollide(self.bullets, self.aliens, True, True)
        
        # Start new level when all aliens are shot down
        if not self.aliens:
                self._start_new_level()
            
        if collisions:
            # Play the sound effect
            self.soundfx.alien_explosion_fx.play()
            
            # Add the bullet explosion instance
            for bullet in collisions.keys():
                explosion = Explosion(self, 'bullet', bullet.rect.centerx, bullet.rect.centery)
                self.bullet_explosions.add(explosion)
            
            # Counts and scores all aliens hit by bullet
            for aliens in collisions.values():
                self.stats.score += self.settings.alien_points * len(aliens)
                self.sb.prep_score()
                self.sb.check_high_score()                   
            
    def _check_power_shot_alien_collisions(self):
        """Check for power shot and alien collisions"""
        # Delete any alien hit by the powershot
        collisions = pygame.sprite.groupcollide(self.power_shots, self.aliens, False, True)
        
        if collisions:
            # Play the sound effect
            self.soundfx.alien_explosion_fx.play()
            
            # Add the power shot explosion instance
            for power_shot in collisions.keys():
                explosion = Explosion(self, 'power_shot', power_shot.rect.x, power_shot.rect.y)
                self.power_shot_explosions.add(explosion)
            
            # Counts and scores all aliens hit by power shot
            for aliens in collisions.values():
                self.stats.score += self.settings.alien_points * len(aliens)
                self.sb.prep_score()
                self.sb.check_high_score()
        
    def _start_new_level(self):
        """Start the new level when whole fleet is destroyed"""
        
        # Delete the remaining bullets and create a new fleet
        self.bullets.empty()
        self._create_fleet()
        self.settings.increase_speed()
            
        # Level increment
        self.stats.level += 1
        self.sb.prep_level()
        
    def _store_high_score(self):
        """Save the highest score recorded"""
        
        filename = 'highscore.json'
        with open(filename, 'w') as file_object:
            json.dump(self.sb.high_score, file_object)
    
    def _update_screen(self):
        """Update images on the screen, and flip to the next screen"""
         
        # Redrawn the screen during each pass through the loop
        self.screen.blit(self.settings.bg, (0,0))
        
        # Draw the ship 
        self.ship.blitme()
        
        # Draw all the sprites
        for bullet in self.bullets.sprites():
            bullet.draw_bullet()
        
        self.aliens.draw(self.screen)    
        
        for power_shot in self.power_shots.sprites():
            power_shot.draw_power_shot()
        
        # Draw all the explosions
        for explosion in self.bullet_explosions.sprites():
            explosion.draw_explosion()
            
        for explosion in self.power_shot_explosions.sprites():
            explosion.draw_explosion()
            
        for explosion in self.ship_explosions.sprites():
            explosion.draw_explosion()
        
        # Draw the scoreboard 
        self.sb.show_score()
        
        # Draw the play button and the difficulty button if the game is inactive
        if not self.stats.game_active:
            if not self.stats.game_pause:
                self.difficulty_button.draw_button()
                self.play_button.draw_button()
        
        # Draw the menu if the game is pause
            elif self.stats.game_pause == True:
                self.resume_button.draw_button()
                self.restart_button.draw_button()
                self.quit_button.draw_button()
        
        # Draw the game over! text if game ends
        if self.stats.game_over == True:
            self.screen.blit(self.sb.game_over_image, self.sb.game_over_rect)
        
        # Make the most recently drawn screen visible
        pygame.display.flip()
        
if __name__ == '__main__':
    # Make a game instance and run the game
    ai = AlienInvasion()
    ai.run_game()

SystemExit: 